In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option("display.max_columns", 85)

# SVM: Scoring by Nested Cross-Validation

### Load the data

In [2]:
# Read Data from CSV (NEW DATA, NOT SCALED)
df = pd.read_csv("../data/abnormal_writeout_noscale.data.csv", index_col=0)

# trascurare da ACC a UVM
start_drop = df.columns.get_loc("ACC")
end_drop = df.columns.get_loc("UVM")
cols = np.arange(start_drop, end_drop + 1)
df.drop(df.columns[cols], axis=1, inplace=True)

# trascurare alcune colonne
df.drop("TTT_freq", axis=1, inplace=True)
df.drop("oldest_phylostratum_factor", axis=1, inplace=True)
df.drop("gc_cds", axis=1, inplace=True)

# Drop NaNs
df.dropna(inplace=True)

# Sort features
resp = df["response"]
occ = df["occ_total_sum"]
age = df["oldest_phylostratum"]
conf = df.drop(labels=["response", "occ_total_sum", "oldest_phylostratum"], axis=1)

# Collect Features and Labels
features_df = pd.DataFrame()
features_df["occ_total_sum"] = occ
features_df["oldest_phylostratum"] = age
features_df = pd.concat([features_df, conf], axis=1)

X = features_df.to_numpy()
y = df["response"].to_numpy()

features_df.head()

,occ_total_sum,oldest_phylostratum,cds_length,dnase_gene,dnase_cds,H3k4me1_gene,H3k4me3_gene,H3k27ac_gene,H3k4me1_cds,H3k4me3_cds,H3k27ac_cds,lamin_gene,repli_gene,nsome_gene,nsome_cds,transcription_gene,repeat_gene,repeat_cds,recomb_gene,AAA_freq,AAC_freq,AAG_freq,AAT_freq,ACA_freq,ACC_freq,ACG_freq,ACT_freq,AGA_freq,AGC_freq,AGG_freq,AGT_freq,ATA_freq,ATC_freq,ATG_freq,ATT_freq,CAA_freq,CAC_freq,CAG_freq,CAT_freq,CCA_freq,CCC_freq,CCG_freq,CCT_freq,CGA_freq,CGC_freq,CGG_freq,CGT_freq,CTA_freq,CTC_freq,CTG_freq,CTT_freq,GAA_freq,GAC_freq,GAG_freq,GAT_freq,GCA_freq,GCC_freq,GCG_freq,GCT_freq,GGA_freq,GGC_freq,GGG_freq,GGT_freq,GTA_freq,GTC_freq,GTG_freq,GTT_freq,TAA_freq,TAC_freq,TAG_freq,TAT_freq,TCA_freq,TCC_freq,TCG_freq,TCT_freq,TGA_freq,TGC_freq,TGG_freq,TGT_freq,TTA_freq,TTC_freq,TTG_freq
1,33,12.0,1488,0.612230,0.758065,0.561429,1.000000,0.216855,0.661290,1.000000,0.198925,0.0,0.041809,0.809254,0.706453,6.798234,0.040516,0.0,0.000000,0.004755,0.008152,0.007473,0.002717,0.011549,0.026495,0.010870,0.008152,0.010190,0.028533,0.019701,0.009511,0.000679,0.006114,0.010870,0.002038,0.009511,0.019022,0.028533,0.007473,0.027174,0.031250,0.025136,0.029891,0.015625,0.027174,0.019701,0.009511,0.007473,0.017663,0.044837,0.013587,0.008832,0.021739,0.031250,0.008152,0.016984,0.033967,0.027853,0.034647,0.023777,0.030571,0.029212,0.013587,0.000679,0.012908,0.027174,0.003397,0.000000,0.008152,0.000000,0.001359,0.008832,0.021739,0.009511,0.010190,0.020380,0.027174,0.029212,0.010870,0.000679,0.013587,0.005435
10,28,1.0,873,0.086769,0.195876,0.657839,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,-0.007148,0.828752,1.097018,0.061963,0.002809,0.0,2.043350,0.025258,0.019518,0.021814,0.024110,0.025258,0.018370,0.003444,0.012629,0.035591,0.009185,0.016073,0.006889,0.016073,0.017222,0.010333,0.033295,0.019518,0.011481,0.020666,0.022962,0.017222,0.008037,0.002296,0.021814,0.003444,0.001148,0.004592,0.002296,0.008037,0.019518,0.022962,0.019518,0.033295,0.013777,0.019518,0.011481,0.014925,0.006889,0.000000,0.012629,0.018370,0.011481,0.017222,0.018370,0.005741,0.008037,0.012629,0.012629,0.012629,0.014925,0.006889,0.017222,0.017222,0.016073,0.005741,0.022962,0.020666,0.012629,0.027555,0.011481,0.021814,0.017222,0.026406
100,36,1.0,1092,0.479295,0.611722,0.851369,0.354628,0.618954,0.754579,0.030220,0.086996,0.0,0.040463,1.249600,1.354306,6.081620,0.028404,0.0,0.868383,0.018727,0.012172,0.023408,0.003745,0.017790,0.024345,0.007491,0.014981,0.024345,0.020599,0.025281,0.011236,0.003745,0.013109,0.019663,0.004682,0.017790,0.016854,0.029963,0.017790,0.034644,0.022472,0.010300,0.028090,0.005618,0.010300,0.014045,0.003745,0.015918,0.015918,0.033708,0.011236,0.014981,0.022472,0.026217,0.009363,0.015918,0.031835,0.007491,0.025281,0.028090,0.029026,0.021536,0.013109,0.008427,0.010300,0.016854,0.003745,0.006554,0.012172,0.005618,0.008427,0.014981,0.016854,0.009363,0.008427,0.014981,0.019663,0.029026,0.010300,0.004682,0.010300,0.004682
1000,126,1.0,2800,0.171524,0.280357,0.554023,0.052420,0.278492,0.270357,0.021429,0.151429,0.0,-0.022495,0.921420,1.382249,2.254471,0.014520,0.0,1.143060,0.022054,0.014823,0.022415,0.024946,0.022054,0.014100,0.006146,0.015546,0.024946,0.016992,0.012292,0.015907,0.013377,0.021330,0.026392,0.017715,0.026392,0.011931,0.027477,0.017354,0.023861,0.016992,0.006508,0.019161,0.005785,0.003977,0.007954,0.003977,0.006146,0.010846,0.025307,0.015907,0.022415,0.022777,0.016269,0.018800,0.015184,0.016992,0.004700,0.014461,0.017354,0.010484,0.010123,0.011931,0.009400,0.007231,0.020607,0.011931,0.013738,0.008315,0.006146,0.016631,0.022054,0.018077,0.004700,0.009038,0.031092,0.019523,0.019523,0.016992,0.016269,0.014100,0.015907
10000,55,1.0,1484,0.143843,0.030997,0.400789,0.106455,0.457949,0.708221,0.030997,0.659704,0.0,-0.000387,0.960747,1.196871,1.080241,0.009545,0.0,4.217000,0.039835,0.015797,0.030220,0.025412,0.024038,0.012363,0.002747,0.019918,0.048077,0.006868,0.015797,0.009615,0.020604,0.009615,0.032280,0.023352,0.019918,0.012363,0.021978,0

***
### Custom PCA

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Columns of confounder variables (highly colinear)
conf_index = 2
conf_cols = np.arange(2, X.shape[1])


class ConfounderPCA(BaseEstimator, TransformerMixin):
    """ 
    Custom PCA transformer for this dataset.
    Applies PCA only to the many collinear confounder 
    variables.
    
    cols - columns to which PCA will be applied.
    
    n_components - same as with the "vanilla" PCA. 
        If 0 < n_components < 1, select the number of 
        components such that the amount of variance that 
        needs to be explained is greater than the 
        percentage specified by n_components.
        
    apply_PCA - if false, simply returns the untransformed data.
    """

    def __init__(self, cols, n_components=None, apply_PCA=True):
        self.n_components = n_components
        self.apply_PCA = apply_PCA
        self.cols = cols
        if self.apply_PCA:
            self.pca = PCA(n_components=self.n_components)

    def fit(self, X, y=None):
        if self.apply_PCA:
            self.pca.fit(X[:, self.cols])
        return self

    def transform(self, X, y=None):
        if self.apply_PCA:
            X_pca = self.pca.transform(X[:, self.cols])
            return np.c_[X[:, :2], X_pca]
        else:
            return X

### Custom Scoring

In [4]:
from sklearn.metrics import auc, make_scorer, precision_recall_curve


def auprc(y_true, y_scores, **kwargs):
    """ Remember to use make_scorer(auprc, needs_proba=True,) """
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_scores)
    # result is sum of the areas under each curve
    return auc(thresholds, precisions[:-1]) + auc(thresholds, recalls[:-1])

### Train-Test Split

In [5]:
from sklearn.model_selection import train_test_split

X = np.ascontiguousarray(X)
y = np.ascontiguousarray(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

*** 
## Nested CV on SVM (Returning Information on the Best Parameter Configurations)


### The Model and its Parameter Space

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Define a parameter space to search
param_grid = {
    "svm__C": np.logspace(-3, 4, 4),
    "svm__class_weight": ['balanced', None],
    "pca__apply_PCA": [True, False],
    "pca__n_components": [0.95, None],
}

# Define the model to be tuned
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", ConfounderPCA(cols=np.arange(2, X.shape[1]))),
    ("svm", SVC(kernel='rbf', probability=True)),
])

### Nested CV

In [7]:
from sklearn.exceptions import ConvergenceWarning, FitFailedWarning
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, RepeatedKFold
from sklearn.utils._testing import ignore_warnings

# configure the cross-validation procedure
k_outer = 10
k_inner = 3
cv_outer = KFold(n_splits=k_outer, shuffle=True, random_state=1)
cv_inner = KFold(n_splits=k_inner, shuffle=True, random_state=3)

# To store results
roc_results = list()
# auprc_results = list()
# prec_results = list()
# rec_results = list()
# f1_results = list()
found_params = list()

print(f"Performing nested-cv with {k_outer} outer-folds and {k_inner} inner-folds.\n")
print("OUTER CV | BEST OF INNER CV | CHOSEN PARAMS")

for train_ix, test_ix in cv_outer.split(X_train):

    # split data
    X_tr, X_te = X_train[train_ix, :], X_train[test_ix, :]
    y_tr, y_te = y_train[train_ix], y_train[test_ix]

    # If some parameter combinations are incompatible:
    # with ignore_warnings(category=[ConvergenceWarning, FitFailedWarning]):
    
    # define search
    search = GridSearchCV(estimator=svm_clf, param_grid=param_grid, scoring="roc_auc", cv=cv_inner, n_jobs=-1)
    # execute search
    result = search.fit(X_tr, y_tr)
        
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_

    # evaluate model on the hold out dataset
    # yhat = best_model.predict(X_te)
    yhat = best_model.predict_log_proba(X_te)[:,1]

    # evaluate the model
    roc_auc = roc_auc_score(y_te, yhat)
    
    # store the result
    roc_results.append(roc_auc)
    # auprc_results.append(auprc(y_te, yhat))
    # prec_results.append(precision_score(y_te, yhat))
    # rec_results.append(recall_score(y_te, yhat))
    # f1_results.append(f1_score(y_te, yhat))
    found_params.append(result.best_params_)

    # report progress
    print(">roc-auc=%.3f, est=%.3f, params=%s" % (roc_auc, result.best_score_, result.best_params_))

# summarize the estimated performance of the model
print("ROC-AUC: %.3f (std = %.3f)" % (np.mean(roc_results), np.std(roc_results)))

Performing nested-cv with 10 outer-folds and 3 inner-folds.

OUTER CV | BEST OF INNER CV | CHOSEN PARAMS
>roc-auc=0.675, est=0.671, params={'pca__apply_PCA': True, 'pca__n_components': 0.95, 'svm__C': 0.21544346900318845, 'svm__class_weight': 'balanced'}
>roc-auc=0.683, est=0.667, params={'pca__apply_PCA': True, 'pca__n_components': 0.95, 'svm__C': 0.21544346900318845, 'svm__class_weight': 'balanced'}
>roc-auc=0.676, est=0.669, params={'pca__apply_PCA': True, 'pca__n_components': 0.95, 'svm__C': 0.21544346900318845, 'svm__class_weight': 'balanced'}
>roc-auc=0.627, est=0.676, params={'pca__apply_PCA': True, 'pca__n_components': 0.95, 'svm__C': 0.21544346900318845, 'svm__class_weight': 'balanced'}
>roc-auc=0.666, est=0.674, params={'pca__apply_PCA': True, 'pca__n_components': 0.95, 'svm__C': 0.21544346900318845, 'svm__class_weight': 'balanced'}
>roc-auc=0.686, est=0.670, params={'pca__apply_PCA': True, 'pca__n_components': 0.95, 'svm__C': 0.21544346900318845, 'svm__class_weight': 'balanc

In [8]:
ncv_df = pd.DataFrame(roc_results, columns=['roc_auc'])
ncv_df = pd.concat([ncv_df, pd.DataFrame(found_params)], axis=1)
ncv_df

,roc_auc,pca__apply_PCA,pca__n_components,svm__C,svm__class_weight
0,0.675144,True,0.95,0.215443,balanced
1,0.682503,True,0.95,0.215443,balanced
2,0.676012,True,0.95,0.215443,balanced
3,0.626791,True,0.95,0.215443,balanced
4,0.665610,True,0.95,0.215443,balanced
5,0.685762,True,0.95,0.215443,balanced
6,0.670468,True,0.95,0.215443,balanced
7,0.694589,True,0.95,0.215443,balanced
8,0.700855,True,0.95,0.215443,balanced
9,0.678854,True,0.95,0.215443,balanced


In [9]:
ncv_df["roc_auc"].mean()

0.6756588317019976

In [10]:
ncv_df.to_csv("./data/svm_ncv.csv")

# Model Scoring on Test Set

In [11]:
# Define the grid search object
gscv = GridSearchCV(
    estimator=svm_clf,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,
    verbose=1,
    scoring="roc_auc", 
)

# Search
gscv_result = gscv.fit(X_train, y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


In [12]:
gs_data = pd.DataFrame(gscv_result.cv_results_)
gs_data.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_pca__apply_PCA,param_pca__n_components,param_svm__C,param_svm__class_weight,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,121.409958,0.279862,10.996073,0.305803,True,0.95,0.001,balanced,"{'pca__apply_PCA': True, 'pca__n_components': ...",0.639595,0.636631,0.627086,0.634437,0.005337,5
1,53.620157,0.081328,4.601591,0.142764,True,0.95,0.001,None,"{'pca__apply_PCA': True, 'pca__n_components': ...",0.549174,0.545711,0.551315,0.548733,0.002309,29
2,102.752787,0.893658,9.286273,0.074044,True,0.95,0.215443,balanced,"{'pca__apply_PCA': True, 'pca__n_components': ...",0.666073,0.676925,0.666410,0.669803,0.005038,1
3,81.969081,1.420197,7.181552,0.280997,True,0.95,0.215443,None,"{'pca__apply_PCA': True, 'pca__n_components': ...",0.604147,0.601403,0.607549,0.604366,0.002514,25
4,130.739825,1.304148,7.371101,0.180032,True,0.95,46.415888,balanced,"{'pca__apply_PCA': True, 'pca__n_components': ...",0.619417,0.614831,0.619045,0.617764,0.002079,9


In [13]:
gs_data.groupby(by='param_pca__apply_PCA').mean()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
param_pca__apply_PCA,,,,,,,,,,
False,109.440788,2.997241,7.243874,0.217717,0.613534,0.611239,0.613502,0.612758,0.003435,14.5
True,112.139282,1.472471,7.529050,0.164035,0.613534,0.611239,0.613502,0.612758,0.003435,14.5


In [14]:
model = gscv_result.best_estimator_
print("Best Params:")
print(gscv_result.best_params_)
model.fit(X_train, y_train)

Best Params:
{'pca__apply_PCA': True, 'pca__n_components': 0.95, 'svm__C': 0.21544346900318845, 'svm__class_weight': 'balanced'}


Pipeline(steps=[('scaler', StandardScaler()),
                ('pca',
                 ConfounderPCA(cols=array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52,
       53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
       70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81]),
                               n_components=0.95)),
                ('svm',
                 SVC(C=0.21544346900318845, class_weight='balanced',
                     probability=True))])

In [15]:
from sklearn.model_selection import cross_val_score

cross_val_score(model, X_test, y_test, cv=3, scoring='roc_auc')

array([0.63796247, 0.64414342, 0.65106188])